In [1]:
import pandas as pd
import numpy as np
import requests
from pprint import pprint
from bs4 import BeautifulSoup
import datetime as dt

import re, string
import nltk
import contractions
from nltk.corpus import stopwords

from api_keys import api_key

In [2]:
def request_song_info(artist_id, page_number):
    base_url = f'https://api.genius.com'
    headers = {f'Authorization': f'Bearer {api_key}'}
    search_url = f'{base_url}/artists/{artist_id}/songs?per_page=40&page={page_number}&sort=popularity'
    response = requests.get(search_url, headers=headers)

    return response

In [3]:
jay_dict = {}
beyonce_dict = {}
destiny_dict = {}
artist_dict = {2 : jay_dict, 498 : beyonce_dict, 2831 : destiny_dict}
        
for artist_id, artist_dict in artist_dict.items():      

    for page in range (1, 20):
        response = request_song_info(artist_id, page)
        json = response.json()
        remote_song_info = None

        for song in json['response']['songs']:
            if str(artist_id) in str(song['primary_artist']['id']):
                remote_song_info = song

                if remote_song_info:
                    song_url = remote_song_info['url']

                if len(artist_dict) == 100:
                    break

                else:
                    artist_dict.update({song['title'].lower() : song_url})

In [4]:
print(f'the jay_z_dict contains {len(jay_dict)} songs.')
print(f'the beyonce_dict contains {len(beyonce_dict)} songs.')
print(f'the destiny_dict contains {len(destiny_dict)} songs.')

the jay_z_dict contains 100 songs.
the beyonce_dict contains 100 songs.
the destiny_dict contains 100 songs.


In [5]:
def scrap_song_url(song_url, date_index):
    page = requests.get(song_url)
    html = BeautifulSoup(page.text, 'html.parser')
    release_date = dt.datetime.strptime(html.find_all('span', {'class': 'metadata_unit-info metadata_unit-info--text_only'})[date_index].get_text(), '%B %d, %Y').strftime('%Y%m%d')
    lyrics = html.find('div', class_='lyrics').get_text()

    return release_date, lyrics

In [6]:
relationship_start = scrap_song_url(jay_dict["'03 bonnie & clyde"], 1)[0]
print(int(relationship_start))

20021010


In [7]:
pre_jay_dict = {}
post_jay_dict = {}

for song, song_url in jay_dict.items():
    try:
        if int(scrap_song_url(song_url, 1)[0]) >= int(relationship_start):
            if len(post_jay_dict) == 7:
                continue
            else:
                post_jay_dict.update({song : [scrap_song_url(song_url, 1)[0], song_url, scrap_song_url(song_url, 1)[1]]})
        else:
            if len(pre_jay_dict) == 7:
                continue
            else:
                pre_jay_dict.update({song : [scrap_song_url(song_url, 1)[0], song_url, scrap_song_url(song_url, 1)[1]]})

    except:
        try:
            if int(scrap_song_url(song_url, 0)[0]) >= int(relationship_start):
                if len(post_jay_dict) == 7:
                    continue
                else:
                    post_jay_dict.update({song : [scrap_song_url(song_url, 0)[0], song_url, scrap_song_url(song_url, 0)[1]]})
            else:
                if len(pre_jay_dict) == 7:
                    continue
                else:
                    pre_jay_dict.update({song : [scrap_song_url(song_url, 0)[0], song_url, scrap_song_url(song_url, 0)[1]]})                
                
        except:
            print(f'FAILED on {song}')

In [8]:
print(len(pre_jay_dict))
print(len(post_jay_dict))

7
7


In [9]:
pre_beyonce_dict = {}
post_beyonce_dict = {}

for song, song_url in beyonce_dict.items():
    try:
        if int(scrap_song_url(song_url, 1)[0]) >= int(relationship_start):
            if len(post_beyonce_dict) == 7:
                continue
            else:
                post_beyonce_dict.update({song : [scrap_song_url(song_url, 1)[0],  song_url, scrap_song_url(song_url, 1)[1]]})
        else:
            if len(pre_beyonce_dict) == 7:
                continue
            else:
                pre_beyonce_dict.update({song : [scrap_song_url(song_url, 1)[0], song_url, scrap_song_url(song_url, 1)[1]]})

    except:
        try:
            if int(scrap_song_url(song_url, 0)[0]) >= int(relationship_start):
                if len(post_beyonce_dict) == 7:
                    continue
                else:
                    post_beyonce_dict.update({song : [scrap_song_url(song_url, 0)[0], song_url, scrap_song_url(song_url, 0)[1]]})
            else:
                if len(pre_beyonce_dict) == 7:
                    continue
                else:
                    pre_beyonce_dict.update({song : [scrap_song_url(song_url, 0)[0], song_url, scrap_song_url(song_url, 0)[1]]})                
                
        except:
            print(f'FAILED on {song}')

FAILED on bow down
FAILED on beyoncé [tracklist]
FAILED on hello
FAILED on roc
FAILED on wake up
FAILED on denial (poem)
FAILED on flaws and all


In [10]:
print(len(pre_beyonce_dict))
print(len(post_beyonce_dict))

0
7


In [11]:
pre_destiny_dict = {}

for song, song_url in destiny_dict.items():
    try:
        if int(scrap_song_url(song_url, 1)[0]) < int(relationship_start):
            if len(pre_destiny_dict) == 7:
                continue
            else:
                pre_destiny_dict.update({song : [scrap_song_url(song_url, 1)[0], song_url, scrap_song_url(song_url, 1)[1]]})
    except:
        try:
            if int(scrap_song_url(song_url, 0)[0]) < int(relationship_start):
                if len(pre_destiny_dict) == 7:
                    continue
                else:
                    pre_destiny_dict.update({song : [scrap_song_url(song_url, 0)[0], song_url, scrap_song_url(song_url, 0)[1]]})
        except:
            print(f'FAILED on {song}')

FAILED on opera of the bells
FAILED on what's it gonna be
FAILED on i can't go for that
FAILED on why you actin'
FAILED on my man
FAILED on silent night
FAILED on jumpin', jumpin' (remix extended version)
FAILED on got's my own
FAILED on love me (not)
FAILED on proud family
FAILED on dot
FAILED on this love
FAILED on a "dc" christmas medley
FAILED on 2 step
FAILED on this christmas
FAILED on winter paradise
FAILED on hold my beer


In [12]:
print(len(pre_destiny_dict))

7


In [61]:
pre_jay_df = pd.DataFrame.from_dict(pre_jay_dict, orient = 'index', columns = ['release_date', 'song_url', 'lyrics', ])
post_jay_df = pd.DataFrame.from_dict(post_jay_dict, orient = 'index', columns = ['release_date', 'song_url', 'lyrics'])
pre_beyonce_df = pd.DataFrame.from_dict(pre_beyonce_dict, orient = 'index', columns = ['release_date', 'song_url', 'lyrics'])
post_beyonce_df = pd.DataFrame.from_dict(post_beyonce_dict, orient = 'index', columns = ['release_date', 'song_url', 'lyrics'])
pre_destiny_df = pd.DataFrame.from_dict(pre_destiny_dict, orient = 'index', columns = ['release_date', 'song_url', 'lyrics'])

In [62]:
stop = stopwords.words('english')

def clean_text(artist_df):
    artist_df = artist_df.apply(lambda x: x.replace('\n', ' '))
    artist_df = artist_df.apply(lambda x: re.sub('\[[^]]*\]', '', x))
    artist_df = artist_df.apply(lambda x: x.lower())
    artist_df = artist_df.apply(lambda x: contractions.fix(x))
    artist_df = artist_df.apply(lambda x: re.sub(r'[^\w\s]','', x))
    artist_df = artist_df.apply(lambda x: [item for item in x.split() if item not in stop])
    return artist_df

In [63]:
pre_jay_df.lyrics = clean_text(pre_jay_df.lyrics)
post_jay_df.lyrics = clean_text(post_jay_df.lyrics)
pre_beyonce_df.lyrics = clean_text(pre_beyonce_df.lyrics)
post_beyonce_df.lyrics = clean_text(post_beyonce_df.lyrics)
pre_destiny_df.lyrics = clean_text(pre_destiny_df.lyrics)

In [64]:
pre_beyonce_df = pd.concat([pre_beyonce_df, pre_destiny_df])

In [65]:
pre_jay_df['unique_words'] = pre_jay_df.lyrics.apply(lambda x: len(set(x)))
pre_jay_df['word_count'] = pre_jay_df.lyrics.apply(lambda x: len(x))
pre_jay_df['percent_unique'] = pre_jay_df.unique_words / pre_jay_df.word_count

post_jay_df['unique_words'] = post_jay_df.lyrics.apply(lambda x: len(set(x)))
post_jay_df['word_count'] = post_jay_df.lyrics.apply(lambda x: len(x))
post_jay_df['percent_unique'] = post_jay_df.unique_words / post_jay_df.word_count

pre_beyonce_df['unique_words'] = pre_beyonce_df.lyrics.apply(lambda x: len(set(x)))
pre_beyonce_df['word_count'] = pre_beyonce_df.lyrics.apply(lambda x: len(x))
pre_beyonce_df['percent_unique'] = pre_beyonce_df.unique_words / pre_beyonce_df.word_count

post_beyonce_df['unique_words'] = post_beyonce_df.lyrics.apply(lambda x: len(set(x)))
post_beyonce_df['word_count'] = post_beyonce_df.lyrics.apply(lambda x: len(x))
post_beyonce_df['percent_unique'] = post_beyonce_df.unique_words / post_beyonce_df.word_count

In [28]:
type(pre_jay_df.iloc[2,0])

str

In [83]:
pre_jay_df['years'] = pd.to_datetime(pre_jay_df['release_date'], format='%Y%m%d') - pd.to_datetime(relationship_start, format= '%Y%m%d')
pre_jay_df['years'] = pre_jay_df['years'].dt.days / 365

post_jay_df['years'] = pd.to_datetime(post_jay_df['release_date'], format='%Y%m%d') - pd.to_datetime(relationship_start, format= '%Y%m%d')
post_jay_df['years'] = post_jay_df['years'].dt.days / 365

pre_beyonce_df['years'] = pd.to_datetime(pre_beyonce_df['release_date'], format='%Y%m%d') - pd.to_datetime(relationship_start, format= '%Y%m%d')
pre_beyonce_df['years'] = pre_beyonce_df['years'].dt.days / 365

post_beyonce_df['years'] = pd.to_datetime(post_beyonce_df['release_date'], format='%Y%m%d') - pd.to_datetime(relationship_start, format= '%Y%m%d')
post_beyonce_df['years'] = post_beyonce_df['years'].dt.days / 365

In [25]:
pre_jay_data = {'release_date' : pre_jay_df.release_date.values.tolist(), 
               'percent_unique' : pre_jay_df.percent_unique.values.tolist(),
               'word_count' : pre_jay_df.word_count.values.tolist(),
               'song': pre_jay_df.index.tolist(),
               'song_url': pre_jay_df.song_url.values.tolist()}

In [26]:
pre_jay_data

{'release_date': [19991013,
  20010911,
  20010911,
  20020508,
  19991228,
  20010911,
  19960624],
 'percent_unique': [0.4954545454545455,
  0.6666666666666666,
  0.6166328600405679,
  0.4928774928774929,
  0.6076555023923444,
  0.5073313782991202,
  0.5579514824797843],
 'word_count': [440, 459, 493, 351, 418, 341, 371],
 'song': [0, 1, 2, 3, 4, 5, 6],
 'song_url': ['https://genius.com/Dr-dre-still-dre-lyrics',
  'https://genius.com/Jay-z-renegade-lyrics',
  'https://genius.com/Jay-z-takeover-lyrics',
  'https://genius.com/Jay-z-song-cry-lyrics',
  'https://genius.com/Jay-z-big-pimpin-lyrics',
  'https://genius.com/Jay-z-izzo-hova-lyrics',
  'https://genius.com/Jay-z-devils-lyrics']}